# 03 Plotting a MACD chart.

* **Original Text**: https://excelsior-cjh.tistory.com/110?category=975542 [EXCELSIOR]

* **Final Edit**: 2021-08-03

In [1]:
!pip3 install --user pandas
!pip3 install --user html5lib
!pip3 install --user BeautifulSoup4
!pip3 install --user plotly
!pip3 install --user nbformat

In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}

def get_url(item_name, code_df): 
  code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
  url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
  print("요청 URL = {}".format(url)) 
  return url

def get_html_table_symbol(url):
  response = requests.get(url, headers=headers)
  html = bs(response.text, 'lxml')
  html_table = html.select('table')
  len(html_table)
  return str(html_table)

# 1: Get item_code from 상장법인목록.xls
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 
code_df = code_df[['회사명', '종목코드']]
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'})

# 2. Get stock data from naver finance
item_name='삼성전자' 
url = get_url(item_name, code_df)

df = pd.DataFrame()
for page in range(1, 21):
  pg_url = '{url}&page={page}'.format(url=url, page=page)
  table = get_html_table_symbol(pg_url)
  df = df.append(pd.read_html(table, header=0)[0], ignore_index=True)
df = df.dropna()

# 3. Rename columns
df = df.rename(columns= {'날짜': 'date', '종가': 'close', '전일비': 'diff', '시가': 'open', '고가': 'high', '저가': 'low', '거래량': 'volume'}) 
df[['close', 'diff', 'open', 'high', 'low', 'volume']] = df[['close', 'diff', 'open', 'high', 'low', 'volume']].astype(int) 
df['date'] = pd.to_datetime(df['date']) 
df = df.sort_values(by=['date'], ascending=True) 

요청 URL = http://finance.naver.com/item/sise_day.nhn?code=005930


In [3]:
def get_macd(df, short=12, long=26, t=9):
  df = pd.DataFrame(df)
  ma_12 = df.close.ewm(span=12).mean()
  ma_26 = df.close.ewm(span=26).mean()
  macd = ma_12 - ma_26
  macds = macd.ewm(span=9).mean()
  macdo = macd - macds

  df = df.assign(macd=macd, macds=macds, macdo=macdo).dropna()
  return df

In [4]:
df = get_macd(df)

In [5]:
from plotly import subplots
import plotly.offline as offline 
import plotly.graph_objs as go 

macd = go.Scatter( x=df.date, y=df['macd'], name="MACD") 
signal = go.Scatter( x=df.date, y=df['macds'], name="Signal") 
oscillator = go.Bar( x=df.date, y=df['macdo'], name="oscillator")
trade_volume = go.Bar( x=df.date, y=df['volume'], name="volume") 
data = [macd, signal, oscillator]

layout = go.Layout(title='{} MACD 그래프'.format(item_name))
fig = subplots.make_subplots(rows=2, cols=1, shared_xaxes=True)

for trace in data: 
  fig.append_trace(trace, 1,1) 

fig.append_trace(trade_volume, 2,1)
#fig = go.Figure(data=data, layout=layout)
offline.iplot(fig)
